## makemore: babynames tflite

In [1]:
from time import time
myseed=round(time())

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [ ]:
words = open('sat_lotto.txt', 'r').read().splitlines()
words[:8]

In [ ]:
len(words)

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(stoi)
print(vocab_size)

In [ ]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

#abigail a b i, g

def build_dataset(words):  
    block_size = 3
    X,Y = [], []
    for w in words:
#         print(w)
        context = [0]*block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
        
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [ ]:
for x,y in zip(Xtr[:20], Ytr[:20]):
#     print(itos[ix.item() for ix in x], '-->', itos[y.item()])
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

In [ ]:
torch.manual_seed(myseed+46); # seed rng for reproducibility

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, BatchNormalization, Dropout
embedding_dim=16
max_sequence_length=block_size
h1 = 256
h2 = 128
# Define the model
model = Sequential([
    # Embedding Layer
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    
    # Flatten the embedded sequences (or you can use GlobalAveragePooling1D, GlobalMaxPooling1D, etc.)
    tf.keras.layers.Flatten(),
    
    # Fully Connected Neural Network (Dense Layer)
    Dense(units=h1, activation='relu'),
    
    # Batch Normalization
    BatchNormalization(),
    
    # Dropout for regularization
    Dropout(0.5),  # Adjust the dropout rate as needed
    
    # Another Fully Connected Neural Network (Dense Layer)
    Dense(units=h2, activation='relu'),
    
    Dropout(0.5),  # Adjust the dropout rate as needed
    
    # Output Layer (depending on your problem)
    Dense(units=vocab_size, activation='softmax')  # Adjust the activation function for your problem
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
model.fit(Xtr.numpy(),Ytr.numpy(), epochs = 10, batch_size = 32, verbose = 1)

In [ ]:
loss, accuracy = model.evaluate(Xtr.numpy(),Ytr.numpy())

In [ ]:
loss, accuracy = model.evaluate(Xdev.numpy(), Ydev.numpy())

In [ ]:
loss, accuracy = model.evaluate(Xte.numpy(), Yte.numpy())

In [ ]:
import math
-math.log(1/27)

In [ ]:
# sample from the model
# import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
import numpy as np

g = torch.Generator().manual_seed(myseed+12)#2147483647 + 12)
start = []

for i in range(20):

    out = []    
    context = [0] * block_size # initialize with all ...
    
    #context = [np.random.randint(1, 46)] * block_size # initialize with all ...
    #out = context
    while True:
        probs = model.predict([context],verbose=0);
        probs = torch.from_numpy(probs)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0: #do until .
            break
    print(''.join(itos[i] for i in out))

In [ ]:
probs = model.predict([context], verbose=0);

In [ ]:
import numpy as np
np.random.randint(1, 46, size =100)

In [ ]:
len(out)